In [41]:
from split_video_gpx import parse_gpx
from geom_helper import pts2segs
from itertools import chain
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
from snap2road import snap2road
from trace2seg import seg_disambiguation
from utils import *

In [57]:
gpx = 'Sample Data/GPX/Track_2017-02-21 125536.gpx'
# gpx = 'Sample Data/GPX/Track_2017-02-21 131933.gpx'

vfile, raw_locs, tms = parse_gpx(gpx)
snapped_res = snap2road(raw_locs,tms)
snapped_df = pd.DataFrame.from_dict(snapped_res['snapped'])
raw_df = pd.DataFrame.from_dict(snapped_res['raw'])


In [4]:
segs = gp.read_file(u'd:\\★★学习工作\\Life in Maryland\\Research Project：CyclingSafe\\Cycling_safety\\open_data_analysis\\data\\segments_dc.geojson')

In [58]:
trace = raw_locs
tms = tms
close_jn_dist=10
far_jn_dist=30
cnsectv_stepsize=3
cnsectv_thres=0.08
# def trace2segs(segs, trace, tms):
"""
segs: gpdf
trace: list of (lon, lat) points
tms: list of "%Y-%m-%dT%H:%M:%SZ"
"""
# snapped_res = snap2road(trace,tms)
# snapped_df = pd.DataFrame.from_dict(snapped_res['snapped'])
snapped_trace = list(chain(*snapped_df.snapped.values))

In [59]:
snapped_trace_gpdf = gp.GeoDataFrame([Point(x) for x in snapped_trace], columns=['geometry'])


In [60]:
pts_segs, _ = pts2segs(snapped_trace_gpdf, segs, bfr_crs=3559, close_jn_dist=close_jn_dist, far_jn_dist=far_jn_dist)


In [61]:
pts_idx_ln_idx = snapped_trace_gpdf.merge(pts_segs, left_index=True, right_on='index_pt').groupby('index_pt')['index_ln'].apply(list).to_frame()
snapped_trace_gpdf = snapped_trace_gpdf.merge(pts_idx_ln_idx, left_index=True, right_index=True)

In [62]:
snapped_trace_gpdf['clean_seg'] = seg_disambiguation(snapped_trace_gpdf.index_ln.values, keep_tie=True)
snapped_trace_gpdf['clean_seg2'] = seg_disambiguation(snapped_trace_gpdf.clean_seg.values)


In [63]:
trace_segs_idx = pd.unique(chain(*snapped_trace_gpdf.clean_seg2.values))
segs_linear_reference = []
for seg_index in trace_segs_idx:
    seg = segs.loc[seg_index, 'geometry']
    projected = snapped_trace_gpdf[snapped_trace_gpdf.index_ln.apply(lambda x: seg_index in x)].geometry.apply(
        lambda x: seg.project(x, normalized=True))
    for sub_indices in group_consecutive(projected.index.values, stepsize=cnsectv_stepsize):
        sub = projected[sub_indices]
        s, e = sub.min(), sub.max()
        round_s, round_e = float_round(s, direction='down'), float_round(e, direction='up')
        if e - s > cnsectv_thres:
            segs_linear_reference.append((seg_index, round_s, round_e, round_e - round_s))


In [64]:
segs_lin = pd.DataFrame(segs_linear_reference, columns=['index', 'start','end', 'diff']).set_index('index')

In [65]:
from shapely.geometry import LineString

In [66]:
snap_line

,geometry,color
0,"LINESTRING (-77.009486 38.902514, -77.00944699...",#aa0


In [67]:
# visualize the trace and corresponding segments
part_segs_dc_plt= segs[['geometry','STREETSEGID']].merge(segs_lin, left_index=True, right_index=True).reset_index()
part_segs_dc_plt['color']='#0a0'

snap_line = gp.GeoDataFrame([LineString(snapped_trace_gpdf.geometry.tolist())], columns=['geometry'])
snap_line['color'] = '#aa0'

snap_pts_gpdf_plt = snapped_trace_gpdf.reset_index()
snap_pts_gpdf_plt['wkt'] = snap_pts_gpdf_plt.geometry.apply(lambda x: x.wkt)
snap_pts_gpdf_plt.index_ln = snap_pts_gpdf_plt.index_ln.apply(str)
snap_pts_gpdf_plt.clean_seg = snap_pts_gpdf_plt.clean_seg.apply(str)
snap_pts_gpdf_plt.clean_seg2 = snap_pts_gpdf_plt.clean_seg2.apply(str)
snap_pts_gpdf_plt.geometry = snap_pts_gpdf_plt.buffer(0.00002)
snap_pts_gpdf_plt['color'] = '#a00'

orgn_pts = gp.GeoDataFrame([Point(x) for x in raw_locs],columns=['geometry'])
orgn_pts_plt = orgn_pts.reset_index()
orgn_pts_plt['wkt'] = orgn_pts_plt.geometry.apply(lambda x: x.wkt)
orgn_pts_plt.geometry = orgn_pts_plt.buffer(0.00002)
orgn_pts_plt['color'] = '#00a'

from leaflet_creation import create_map_visualization

html_title = 'openstreetmap elements'
file_path = ''
file_name = 'vis snap2segs2'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['ly1','snap'],['ly2','orgn'],['ly3', 'segments'],['ly4','snap line']]
gpdfs = [snap_pts_gpdf_plt, orgn_pts_plt, part_segs_dc_plt, snap_line]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)